<a href="https://colab.research.google.com/github/SwastikMethi/ResNet-50-vs.-VGG-16/blob/main/Resnet50.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import tarfile
import urllib.request

# Download the Stanford Dogs dataset
url = 'http://vision.stanford.edu/aditya86/ImageNetDogs/images.tar'
dataset_dir = '/content/stanford-dogs'

if not os.path.exists(dataset_dir):
    os.makedirs(dataset_dir)
    urllib.request.urlretrieve(url, os.path.join(dataset_dir, 'images.tar'))

# Extract the dataset
with tarfile.open(os.path.join(dataset_dir, 'images.tar'), 'r') as tar_ref:
    tar_ref.extractall(dataset_dir)

import shutil
import random

# Create train and val directories
train_dir = os.path.join(dataset_dir, 'train')
val_dir = os.path.join(dataset_dir, 'val')
os.makedirs(train_dir, exist_ok=True)
os.makedirs(val_dir, exist_ok=True)

# Split the data into train and validation sets (80% train, 20% val)
for breed_dir in os.listdir(os.path.join(dataset_dir, 'Images')):
    breed_path = os.path.join(dataset_dir, 'Images', breed_dir)
    images = os.listdir(breed_path)
    random.shuffle(images)
    split_index = int(len(images) * 0.8)

    for image in images[:split_index]:
        src = os.path.join(breed_path, image)
        dst = os.path.join(train_dir, breed_dir, image)
        os.makedirs(os.path.dirname(dst), exist_ok=True)
        shutil.move(src, dst)

    for image in images[split_index:]:
        src = os.path.join(breed_path, image)
        dst = os.path.join(val_dir, breed_dir, image)
        os.makedirs(os.path.dirname(dst), exist_ok=True)
        shutil.move(src, dst)

data_dir = dataset_dir

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import os

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define the data transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

data_dir = '/content/stanford-dogs'
batch_size = 16

image_datasets = {
    x: datasets.ImageFolder(
        os.path.join(data_dir, x),
        transform=data_transforms[x]
    )
    for x in ['train', 'val']
}

dataloaders = {
    x: DataLoader(
        image_datasets[x],
        batch_size=batch_size,
        shuffle=True,
        num_workers=0
    )
    for x in ['train', 'val']
}

model = models.resnet50(pretrained=True)

num_classes = 120
model.fc = nn.Linear(model.fc.in_features, num_classes)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

num_epochs = 5
best_acc = 0.0

# Training loop
for epoch in range(num_epochs):
    print(f"Epoch {epoch+1}/{num_epochs}")

    # Training phase
    model.train()
    running_loss, running_corrects = 0.0, 0
    for batch_idx, (inputs, labels) in enumerate(dataloaders['train']):
        inputs, labels = inputs.to(device), labels.to(device)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Track loss and accuracy
        running_loss += loss.item() * inputs.size(0)
        running_corrects += (outputs.argmax(1) == labels).sum().item()

        if batch_idx % 100 == 0:
            print(f"Batch {batch_idx}/{len(dataloaders['train'])}, Loss: {loss.item():.4f}")

    # Calculate epoch statistics
    epoch_loss = running_loss / len(image_datasets['train'])
    epoch_acc = running_corrects / len(image_datasets['train'])
    print(f"Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}")

    model.eval()
    val_loss, val_corrects = 0.0, 0
    with torch.no_grad():
        for inputs, labels in dataloaders['val']:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * inputs.size(0)
            val_corrects += (outputs.argmax(1) == labels).sum().item()

    val_loss /= len(image_datasets['val'])
    val_acc = val_corrects / len(image_datasets['val'])
    print(f"Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}")

    torch.cuda.empty_cache()

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), 'best_resnet50_stanford_dogs.pth')

print("Training complete. Best validation accuracy: {:.4f}".format(best_acc))


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 185MB/s]


Epoch 1/5
Batch 0/1027, Loss: 4.9045
Batch 100/1027, Loss: 4.1261
Batch 200/1027, Loss: 3.0975
Batch 300/1027, Loss: 2.3125
Batch 400/1027, Loss: 2.3180
Batch 500/1027, Loss: 2.0766
Batch 600/1027, Loss: 2.0599
Batch 700/1027, Loss: 1.4294
Batch 800/1027, Loss: 1.6595
Batch 900/1027, Loss: 0.9845
Batch 1000/1027, Loss: 1.4939
Train Loss: 2.3577 Acc: 0.4865
Val Loss: 0.6974 Acc: 0.7979
Epoch 2/5
Batch 0/1027, Loss: 1.2627
Batch 100/1027, Loss: 1.5003
Batch 200/1027, Loss: 1.5517
Batch 300/1027, Loss: 1.6918
Batch 400/1027, Loss: 1.2557
Batch 500/1027, Loss: 1.4436
Batch 600/1027, Loss: 1.4719
Batch 700/1027, Loss: 0.6840
Batch 800/1027, Loss: 1.2666
Batch 900/1027, Loss: 1.3102
Batch 1000/1027, Loss: 0.9961
Train Loss: 1.2880 Acc: 0.6624
Val Loss: 0.5837 Acc: 0.8224
Epoch 3/5
Batch 0/1027, Loss: 1.0526
Batch 100/1027, Loss: 0.9019
Batch 200/1027, Loss: 0.6833
Batch 300/1027, Loss: 1.0359
Batch 400/1027, Loss: 0.8238
Batch 500/1027, Loss: 1.3628
Batch 600/1027, Loss: 1.9129
Batch 700/102